# 要約 
このJupyter Notebookでは、Kaggleの「20の質問」ゲームのエピソードデータを収集し、分析することを目的としています。具体的には、各エピソードでの質問、回答、予測、及び関連するキーワードを取得し、それらのデータをデータフレームに整理して保存する手法が採用されています。

### 問題のアプローチ
1. **データ収集**:
   - KaggleのURLからエピソードのJSONデータを取得するために、`requests`ライブラリを使用しています。
   - 複数のエピソードIDに基づき、各エピソードのURLを生成し、そこからデータを収集します。

2. **データ処理**:
   - 収集したデータから質問、回答、予測、キーワードを抽出し、これらをリストに格納します。
   - 最終的に、リストを利用してPandasのデータフレームを作成し、CSVファイルとして保存します。

3. **データ分析**:
   - 保存されたデータを基に、最も頻繁に聞かれた質問や、キーワードの発見に貢献した質問の頻度を分析します。
   - 分析結果から、上位10件の質問を表示します。

### 使用されるライブラリ
- `requests`: HTTPリクエストを行うためのライブラリ。
- `pandas`: データ操作と解析のための強力なライブラリで、データフレームの作成やCSVファイルへの保存を行います。

### 結果
ノートブックは、エピソードから正常にデータを抽出し、`20_questions_data.csv`というCSVファイルに保存したことを報告し、また、各質問の頻度や効率的な質問のリストも表示します。

この取り組みにより、わかりやすくデータを整理するだけでなく、ゲームにおける質問戦略の分析も可能となります。

---


# 用語概説 
以下は、提供されたJupyter Notebookの内容に基づいて、初心者がつまずきそうな専門用語の簡単な解説です。特に、あまり馴染みがない用語や実務を経験していないと理解しにくいものを中心にしています。

1. **HTTPリクエスト**:
   ウェブサーバーに情報を要求するためのプロトコル。`GET`はサーバーからデータを取得するためのリクエストの一種。

2. **JSON (JavaScript Object Notation)**:
   データを軽量なテキスト形式で表現するためのフォーマット。データの受け渡しやAPI通信でよく使われる。

3. **データフレーム**:
   Pandasライブラリで提供される、2次元のラベル付きデータ構造。エクセルの表のような形式でデータを扱うことができる。

4. **キーワード**:
   データ分析や質問応答システムなどで特定の情報を示す言葉。ここではターゲットとなる言葉を指す。

5. **観察データ (Observation Data)**:
   何らかの実験や観察から得られるデータ。通常、変数の値や条件を含む。

6. **イテレーション (Iteration)**:
   特定のタスクを繰り返し実行するプロセス。ここでは、各エピソードのデータを処理する回数を指している。

7. **フィルタリング (Filtering)**:
   与えられた条件に基づいてデータを選別するプロセス。特定の条件を満たすデータのみを抽出することを指す。

8. **頻度カウント**:
   特定のデータの出現回数を数えること。データの傾向を把握するために使用される。

9. **予測 (Guess)**:
   提示された情報に基づいて、何らかの事象や結果を見込む行為。ここでは、質問に対する回答者の想定する単語を指す。

10. **エピソード (Episode)**:
    ここでは、特定の一連のイベントや試行を指す。コンペティションのルールに従ったゲームのプレイ単位を表す。

これらの用語は、深層学習や機械学習の実務において頻繁に使用されることが多く、特に初心者にとっては理解が難しい場合があります。この解説が役に立つことを願っています。

---


In [ ]:
# 必要なライブラリをインストールします
# 'requests'はHTTPリクエストを簡単に行うためのライブラリです
# 'pandas'はデータ操作や解析のための強力なライブラリです
pip install requests pandas

In [ ]:
import requests  # URLからデータを取得するためのライブラリをインポートします
import pandas as pd  # データフレームを操作するためのライブラリをインポートします

# URLからJSONデータを読み込む関数
def read_json_from_url(url):
    response = requests.get(url)  # 指定したURLにGETリクエストを送信します
    response.raise_for_status()  # 不正なレスポンスがあった場合は例外を発生させて知らせます
    return response.json()  # レスポンスからJSONデータを取得します

# データを格納するリスト
data = []

# ベースURL
base_url = "https://www.kaggleusercontent.com/episodes/"

# 各エピソードIDでループします
for i in [55235305, 55177813, 55181874, 55054770]:
    url = f"{base_url}{i}.json"  # 各エピソードのURLを作成します
    json_data = read_json_from_url(url)  # URLからJSONデータを読み込みます
    # キーと値のペアをイテレートします
    auxVar1 = json_data['steps'][len(json_data['steps'])-1]  # 最後のステップを取得します
    for i in range(len(auxVar1)):  # 最後のステップの各要素をループします
        obs = dict(auxVar1[i])['observation']  # 観察データを取得します
        if 'questions' in obs:  # 'questions'が存在する場合
            for j in range(min(len(obs['questions']),len(obs['guesses']))):  # 質問と予測の最小長さでループします
                data.append({  # データを追加します
                    'iteration' : i+1,  # 何回目のイテレーションか
                    'episode_id': json_data['info']['EpisodeId'],  # エピソードID
                    'questions' : obs['questions'][j],  # 質問
                    'answers'   : obs['answers'][j],  # 答え
                    'guesses'   : obs['guesses'][j],  # 予測
                    'keyword'   : obs['keyword']  # キーワード
                })

# DataFrameを作成します
df = pd.DataFrame(data)  # リストからデータフレームを作成します
df.to_csv('20_questions_data.csv', index=False)  # データフレームをCSVファイルに保存します

print("データが正常に抽出され、'20_questions_data.csv'に保存されました。")  # 処理完了のメッセージを表示します

In [ ]:
# データフレームの最初の数行を表示します
print(df.head())

# 各質問の頻度をカウントします
question_counts = df['questions'].value_counts()

# 最も頻繁に聞かれた質問を表示します
print("最も頻繁に聞かれた質問:")
print(question_counts.head(10))  # 上位10件を表示します

# 予測がキーワードと一致する行をフィルタリングします
keyword_discovered = df[df['guesses'] == df['keyword']]

# キーワードの発見につながった質問の頻度をカウントします
discovery_question_counts = keyword_discovered['questions'].value_counts()

# キーワードを発見するための最も効果的な質問を表示します
print("キーワードを発見するための質問:")
print(discovery_question_counts.head(10))  # 上位10件を表示します